# Models — Training, Tuning, and Evaluation

**Objective**  
Train and evaluate regression models to predict **trip_duration_minutes** (ETA) for Zone-237 (and/or overall) trips. This notebook handles feature assembly, model selection, hyperparameter tuning, evaluation (including robust metrics and optional prediction intervals), and artifact export for downstream use.

**What this notebook does**  
- Loads prepared datasets from `.npz` artifacts  
- Builds feature matrices (numeric + encoded categoricals) and a target vector  
- Splits data into train/validation/test (time-aware or KFold)  
- Trains one or more baseline and advanced regressors  
- Performs hyperparameter tuning (Grid/Random/Bayes; as implemented)  
- Evaluates using **MAE / MedAE / RMSE / R²**, plus optional **CVaR@90(abs err)** and coverage of conformal intervals  
- Saves trained model(s), metrics, feature importances, and predictions

---

## Inputs

- **Primary dataset**:  
  - `npz/intra_zone_trips_237_osm_features.npz` *(NumPy .npz; loaded as pandas DataFrame)*  
    - Required columns (dtypes):  
      - `trip_duration_minutes` — `float` (minutes) **[target]**  
      - Time features: `hour:int` (0–23), `day_of_week:int` (0–6) *(if present)*  
      - Zone/ID features: `PUlocationID:Int64`, `DOlocationID:Int64` *(if present)*  
      - OSM features: `route_len_m:float`, `route_time_min:float`, optional `pu_highway:str`, `do_highway:str`, `pu_lanes:float`, `do_lanes:float`  
      - Optional coords used only for mapping (not fed to tree models by default): `gen_pu_lat`, `gen_pu_lon`, `gen_do_lat`, `gen_do_lon` (`float`, WGS84)

---

## Features & Target

- **Target**: `y = trip_duration_minutes` (`float`)  
- **Design matrix**: `X` assembled from available columns, e.g.:  
  - Numeric: `route_len_m`, `route_time_min`, `hour`, `day_of_week`, `pu_lanes`, `do_lanes`  
  - Categorical (One-Hot/Ordinal): `pu_highway`, `do_highway`, `Affiliated_base_number` *(if present)*  
  - IDs (optional): `PUlocationID`, `DOlocationID` *(encode if used)*  
- **Shapes**:  
  - `X_train`: `(n_train, n_features)`; `y_train`: `(n_train,)`  
  - `X_valid` / `X_test` similar

---

## Outputs 

- **Models** (`models/`):  
  - `models/model_<name>.joblib` *(scikit-learn / XGBoost / LightGBM)*  
  - *(optional)* `models/model_<name>.pt` *(PyTorch, if LSTM used)*
- **Metrics & reports** (`metrics/`):  
  - `metrics/<name>_summary.json` — MAE, MedAE, RMSE, R², optional CVaR@90, PI coverage/width  
  - `metrics/<name>_feature_importance.csv` — per-feature importance (if supported)  
- **Predictions** (`preds/`):  
  - `preds/<name>_test_predictions.csv` — columns: `y_true:float`, `y_pred:float`, optional `pi_low:float`, `pi_high:float`  
- **Figures** (`figs/`) *(optional)*:  
  - Residual plots, calibration curves, importance bars, error-by-bucket charts

---

## ML models used

- **Tree-based regressors**  
  - *RandomForestRegressor (sklearn)*  
  - *GradientBoostingRegressor (sklearn)*  
  - *(optional)* *XGBoostRegressor* or *LightGBMRegressor* (if installed)
- **Quantile / intervals (optional)**  
  - *Quantile Regression Forests*, *Gradient Boosting (loss="quantile")*, or *Conformal Prediction* wrapper for PI coverage targets (e.g., 90%)

---

## Evaluation details

- **Primary metrics**: MAE, MedAE, RMSE, R²  
- **Robust metric** *(optional)*: CVaR@90 of absolute error (tail risk)  
- **Prediction intervals** *(optional)*: 90% coverage target; report empirical coverage and average width  
- **Error slices** *(optional)*: by `duration_cluster` ∈ {short, medium, long}

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import sys, os
sys.path.append('/content/drive/MyDrive/Model')

import npz_io
df = npz_io.load_df_npz('/content/drive/MyDrive/Model/intra_zone_trips_237_osm_features.npz')
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,trip_duration_minutes,hour_of_day,day_of_week,...,dropoff_node,pu_node,do_node,route_len_m,route_time_min,turns,pu_highway,do_highway,pu_lanes,do_lanes
64374,B01413,2025-02-02 14:50:12,2025-02-02 15:03:55,237.0,237.0,0,B01413,13.716667,14,6,...,596788016,42447230,596788016,830.484624,1.238497,2.0,secondary,primary,1.0,3.0
123113,B03614,2025-02-03 14:57:49,2025-02-03 15:08:27,237.0,237.0,0,B03404,10.633333,14,0,...,42431678,42438798,42431678,412.466494,0.615109,0.0,secondary,secondary,3.0,3.0
194419,B03614,2025-02-04 17:45:44,2025-02-04 18:00:57,237.0,237.0,0,B03380,15.216667,17,1,...,42429976,42436477,42429976,1032.478096,1.539729,3.0,secondary,secondary,4.0,3.0
260573,B00628,2025-02-05 18:13:28,2025-02-05 18:14:36,237.0,237.0,0,B00628,1.133333,18,2,...,42430060,42446013,42430060,392.994603,0.586071,1.0,secondary,residential,3.0,NaN
276552,B01087,2025-02-06 07:26:52,2025-02-06 07:30:35,237.0,237.0,0,B01087,3.716667,7,3,...,42429688,42431671,42429688,623.639357,0.930030,2.0,residential,secondary,NaN,3.0


### Tree-based Models ###

Random Forest

In [3]:
import numpy as np, pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, f1_score, confusion_matrix
from sklearn.inspection import permutation_importance
import joblib

TARGET_MODE = "regression"

df = df.copy()

# Ensure/derive essentials
# route_length_km
if "route_length_km" not in df.columns:
    if "route_len_m" in df.columns:
        df["route_length_km"] = df["route_len_m"] / 1000.0
    elif "straight_km" in df.columns:
        df["route_length_km"] = df["straight_km"]
    else:
        raise ValueError("Need route_len_m or straight_km to derive route_length_km.")

# time features
if "pickup_datetime" in df.columns:
    df["pickup_datetime"] = pd.to_datetime(df["pickup_datetime"], errors="coerce")
    if "hour_of_day" not in df.columns:
        df["hour_of_day"] = df["pickup_datetime"].dt.hour
    if "day_of_week" not in df.columns:
        df["day_of_week"] = df["pickup_datetime"].dt.dayofweek

# classification label (if missing)
if TARGET_MODE == "classification" and "duration_cluster" not in df.columns:
    bins = [0, 5, 30, np.inf]; labels = ["short_trip","medium_trip","long_trip"]
    df["duration_cluster"] = pd.cut(df["trip_duration_minutes"], bins=bins, labels=labels, right=False, include_lowest=True)

In [5]:
numeric_candidates = [
    "route_length_km", "straight_km", "turns", "pu_lanes", "do_lanes",
    "hour_of_day", "day_of_week", "bearing_deg", "route_time_min"
]
categorical_candidates = [
    "pu_highway", "do_highway"
]

num_feats = [c for c in numeric_candidates if c in df.columns]
cat_feats = [c for c in categorical_candidates if c in df.columns]

if TARGET_MODE == "regression":
    target_col = "trip_duration_minutes"
    if target_col not in df.columns:
        raise ValueError("trip_duration_minutes not found for regression target.")
    y = pd.to_numeric(df[target_col], errors="coerce")
elif TARGET_MODE == "classification":
    target_col = "duration_cluster"
    if target_col not in df.columns:
        raise ValueError("duration_cluster not found for classification target.")
    y = df[target_col].astype("category")
else:
    raise ValueError("TARGET_MODE must be 'regression' or 'classification'.")

X = df[num_feats + cat_feats].copy()

# Drop rows with missing target
mask = y.notna()
X, y = X.loc[mask], y.loc[mask]

In [6]:
# Split
strat = y if TARGET_MODE == "classification" else None
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=strat
)

In [7]:
# Preprocess
numeric_tf = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
])
categorical_tf = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_tf, num_feats),
        ("cat", categorical_tf, cat_feats)
    ],
    remainder="drop"
)

In [8]:
# Model
if TARGET_MODE == "regression":
    model = RandomForestRegressor(
        n_estimators=400,
        max_depth=None,
        min_samples_leaf=2,
        n_jobs=-1,
        random_state=42
    )
else:
    model = RandomForestClassifier(
        n_estimators=400,
        max_depth=None,
        min_samples_leaf=2,
        class_weight="balanced",
        n_jobs=-1,
        random_state=42
    )

pipe = Pipeline(steps=[("prep", preprocess), ("rf", model)])

In [9]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['route_length_km', 'turns',
                                                   'pu_lanes', 'do_lanes',
                                                   'hour_of_day', 'day_of_week',
                                                   'route_time_min']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['pu_highway',
                                                   'do_highway'])])),
                ('rf',
                 RandomForestRegressor(min_samples_leaf=2, n_estimators=400,
                                       n_jobs=-1, random_state=42))])

In [13]:
# Evaluate
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

pred = pipe.predict(X_test)
mae = mean_absolute_error(y_test, pred)

try:
    rmse = mean_squared_error(y_test, pred, squared=False)
except TypeError:
    rmse = np.sqrt(mean_squared_error(y_test, pred))

r2 = r2_score(y_test, pred)
print(f"[Regression] MAE={mae:.3f} min | RMSE={rmse:.3f} min | R^2={r2:.3f}")

[Regression] MAE=25.119 min | RMSE=38.401 min | R^2=-0.064


Possible Better Performance After Tuning

In [17]:
import numpy as np, pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

dfm = df.copy()

# Sanity check: make sure route_length_km exists and is in KM
if "route_length_km" not in dfm.columns and "route_len_m" in dfm.columns:
    dfm["route_length_km"] = dfm["route_len_m"]/1000.0
assert "route_length_km" in dfm.columns, "Need route_length_km or route_len_m"

# Baselines: mean and simple speed baseline (~20 km/h city)
y = pd.to_numeric(dfm["trip_duration_minutes"], errors="coerce")
X_len = dfm["route_length_km"].values
mean_baseline = np.full_like(y.dropna(), y.mean())
speed_kph = 20.0
speed_baseline = (X_len[y.notna()] / speed_kph) * 60  # minutes
print("Baseline MAE (predict mean):", np.abs(y.dropna()-mean_baseline).mean())
print("Baseline MAE (~20kph):", np.abs(y.dropna()-speed_baseline).mean())

# Features
num_feats = [c for c in ["route_length_km","straight_km","turns","pu_lanes","do_lanes",
                         "hour_of_day","day_of_week","bearing_deg","route_time_min"] if c in dfm.columns]
cat_feats = [c for c in ["pu_highway","do_highway"] if c in dfm.columns]
X = dfm[num_feats + cat_feats].copy()
mask = y.notna()
X, y = X.loc[mask], y.loc[mask]

# Split (random first; try time-based later)
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocess
pre = ColumnTransformer([
    ("num", SimpleImputer(strategy="median"), num_feats),
    ("cat", Pipeline([("imp", SimpleImputer(strategy="most_frequent")),
                      ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False))]), cat_feats)
])

rf = RandomForestRegressor(random_state=42, n_jobs=-1)
pipe = Pipeline([("prep", pre), ("rf", rf)])

# Lightweight tuning
param_dist = {
    "rf__n_estimators": [300, 500, 800],
    "rf__max_depth": [None, 15, 25, 35],
    "rf__min_samples_leaf": [1, 2, 4],
    "rf__max_features": ["sqrt", 0.5, 0.8]
}
search = RandomizedSearchCV(pipe, param_distributions=param_dist, n_iter=15,
                            scoring="neg_mean_absolute_error", cv=3,
                            random_state=42, n_jobs=-1, verbose=0)
search.fit(Xtr, ytr)
best = search.best_estimator_

pred = best.predict(Xte)
mae = mean_absolute_error(yte, pred)
rmse = np.sqrt(((yte - pred)**2).mean())
r2 = r2_score(yte, pred)
print(f"[RF tuned] MAE={mae:.2f} min | RMSE={rmse:.2f} min | R²={r2:.3f}")


Baseline MAE (predict mean): 23.530840410741945
Baseline MAE (~20kph): 17.819980989585925
[RF tuned] MAE=23.54 min | RMSE=37.02 min | R²=0.012


In [18]:
import numpy as np, pandas as pd
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

dfc = df.copy()

# Baseline time (best-case)
if "route_time_min" in dfc.columns and dfc["route_time_min"].notna().any():
    base_min = dfc["route_time_min"].copy()
else:
    # fallback: 20 km/h heuristic
    if "route_length_km" not in dfc and "route_len_m" in dfc:
        dfc["route_length_km"] = dfc["route_len_m"]/1000.0
    base_min = (dfc["route_length_km"] / 20.0) * 60.0

dfc["base_min"] = base_min
dfc["factor"] = dfc["trip_duration_minutes"] / dfc["base_min"]

# clip extreme factors to stabilize (e.g., 0.5× to 5×)
dfc["factor_clipped"] = dfc["factor"].clip(0.5, 5.0)

# Features for congestion factor
num_feats = [c for c in ["hour_of_day","day_of_week","turns","route_length_km","bearing_deg",
                         "pu_lanes","do_lanes"] if c in dfc.columns]
cat_feats = [c for c in ["pu_highway","do_highway"] if c in dfc.columns]
X = dfc[num_feats + cat_feats].copy()
y = dfc["factor_clipped"]

mask = y.replace([np.inf, -np.inf], np.nan).notna() & dfc["base_min"].notna()
X, y = X.loc[mask], y.loc[mask]

Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, random_state=42)

pre = ColumnTransformer([
    ("num", SimpleImputer(strategy="median"), num_feats),
    ("cat", Pipeline([("imp", SimpleImputer(strategy="most_frequent")),
                      ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False))]), cat_feats)
])

hgb = HistGradientBoostingRegressor(
    max_depth=None,
    max_leaf_nodes=31,
    learning_rate=0.08,
    min_samples_leaf=50,
    l2_regularization=0.0,
    random_state=42
)

pipe = Pipeline([("prep", pre), ("hgb", hgb)])
pipe.fit(Xtr, ytr)

# factor predictions
fhat = pipe.predict(Xte)
# final duration prediction
y_hat = (dfc.loc[Xte.index, "base_min"].values * fhat)

y_true = dfc.loc[Xte.index, "trip_duration_minutes"].values
mae = mean_absolute_error(y_true, y_hat)
rmse = np.sqrt(((y_true - y_hat)**2).mean())
r2 = r2_score(y_true, y_hat)
print(f"[Factor model] MAE={mae:.2f} min | RMSE={rmse:.2f} min | R²={r2:.3f}")


[Factor model] MAE=15.83 min | RMSE=37.18 min | R²=-0.174


HistGradientBoostingRegressor

In [20]:
import numpy as np, pandas as pd
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, r2_score, median_absolute_error

dfr = df.copy()

# Build base_min (best-case)
if "route_time_min" in dfr and dfr["route_time_min"].notna().any():
    dfr["base_min"] = dfr["route_time_min"]
else:
    # fallback from distance @ 20 km/h
    if "route_length_km" not in dfr and "route_len_m" in dfr:
        dfr["route_length_km"] = dfr["route_len_m"] / 1000.0
    dfr["base_min"] = (dfr["route_length_km"] / 20.0) * 60.0

# Target & mask
dfr["pickup_datetime"] = pd.to_datetime(dfr["pickup_datetime"], errors="coerce")
mask = dfr["trip_duration_minutes"].notna() & dfr["base_min"].replace([0, np.inf], np.nan).notna()

dfr = dfr.loc[mask].copy()
dfr["y_log_resid"] = np.log(dfr["trip_duration_minutes"].clip(lower=0.5)) - np.log(dfr["base_min"].clip(lower=0.5))

# Features
num_feats = [c for c in ["route_length_km","turns","pu_lanes","do_lanes",
                         "hour_of_day","day_of_week","bearing_deg"] if c in dfr.columns]
cat_feats = [c for c in ["pu_highway","do_highway"] if c in dfr.columns]
X = dfr[num_feats + cat_feats]
y = dfr["y_log_resid"]

# Time-based split: last 20% by pickup_datetime
if "pickup_datetime" in dfr.columns and dfr["pickup_datetime"].notna().any():
    dfr = dfr.sort_values("pickup_datetime")
    split_idx = int(len(dfr) * 0.8)
    tr_idx = dfr.index[:split_idx]; te_idx = dfr.index[split_idx:]
    X_train, X_test = X.loc[tr_idx], X.loc[te_idx]
    y_train, y_test = y.loc[tr_idx], y.loc[te_idx]
    base_train = dfr.loc[tr_idx, "base_min"].values
    base_test  = dfr.loc[te_idx, "base_min"].values
    y_true_test = dfr.loc[te_idx, "trip_duration_minutes"].values
else:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
    base_train = dfr.loc[X_train.index, "base_min"].values
    base_test  = dfr.loc[X_test.index, "base_min"].values
    y_true_test = dfr.loc[X_test.index, "trip_duration_minutes"].values

# Preprocess & model
pre = ColumnTransformer([
    ("num", SimpleImputer(strategy="median"), num_feats),
    ("cat", Pipeline([("imp", SimpleImputer(strategy="most_frequent")),
                      ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False))]), cat_feats)
])

hgb = HistGradientBoostingRegressor(
    learning_rate=0.08,
    max_leaf_nodes=31,
    min_samples_leaf=50,
    random_state=42
)

pipe = Pipeline([("prep", pre), ("hgb", hgb)])
pipe.fit(X_train, y_train)

# Predict duration from log-residual
resid_hat = pipe.predict(X_test)
yhat = base_test * np.exp(resid_hat)

# Metrics
mae  = mean_absolute_error(y_true_test, yhat)
rmse = np.sqrt(np.mean((y_true_test - yhat)**2))
r2   = r2_score(y_true_test, yhat)
medae = median_absolute_error(y_true_test, yhat)
print(f"[Log-residual model] MAE={mae:.2f} min | MedAE={medae:.2f} | RMSE={rmse:.2f} | R²={r2:.3f}")

# Bucket MAE (short/medium/long)
bins = [0, 5, 30, np.inf]; labels = ["short","medium","long"]
dur_bucket = pd.cut(y_true_test, bins=bins, labels=labels, right=False, include_lowest=True)
for lab in labels:
    m = dur_bucket == lab
    if m.any():
        print(f"  {lab:6s} MAE = {mean_absolute_error(y_true_test[m], yhat[m]):.2f} (n={m.sum()})")

[Log-residual model] MAE=14.20 min | MedAE=3.97 | RMSE=32.93 | R²=-0.081
  short  MAE = 5.27 (n=229)
  medium MAE = 4.58 (n=353)
  long   MAE = 80.54 (n=82)


### Robust Evaluation + Robust Predict ###

In [23]:
dfr = df.copy()

# Ensure route_length_km exists
if "route_length_km" not in dfr.columns:
    if "route_len_m" in dfr.columns:
        dfr["route_length_km"] = dfr["route_len_m"] / 1000.0
    elif "straight_km" in dfr.columns:
        dfr["route_length_km"] = dfr["straight_km"]
    else:
        raise ValueError("Need 'route_length_km' (or 'route_len_m' / 'straight_km').")

# Time features
if "pickup_datetime" in dfr.columns:
    dfr["pickup_datetime"] = pd.to_datetime(dfr["pickup_datetime"], errors="coerce")
if "hour_of_day" not in dfr.columns:
    dfr["hour_of_day"] = dfr["pickup_datetime"].dt.hour
if "day_of_week" not in dfr.columns:
    dfr["day_of_week"] = dfr["pickup_datetime"].dt.dayofweek
dfr["hour_of_week"] = dfr["day_of_week"] * 24 + dfr["hour_of_day"]

# Best-case time (base)
if "route_time_min" in dfr.columns and dfr["route_time_min"].notna().any():
    dfr["base_min"] = dfr["route_time_min"]
else:
    dfr["base_min"] = (dfr["route_length_km"] / 20.0) * 60.0  # 20 kph fallback

# Keep rows with valid target/base
mask = dfr["trip_duration_minutes"].notna() & dfr["base_min"].replace([0, np.inf], np.nan).notna()
dfr = dfr.loc[mask].copy()

# Log-residual target (median-friendly)
dfr["y_log_resid"] = (
    np.log(np.clip(dfr["trip_duration_minutes"], 0.5, None))
    - np.log(np.clip(dfr["base_min"], 0.5, None))
)

# Features (use what you have)
num_feats_all = ["route_length_km","turns","pu_lanes","do_lanes",
                 "hour_of_day","day_of_week","bearing_deg"]
cat_feats_all = ["pu_highway","do_highway"]
num_feats = [c for c in num_feats_all if c in dfr.columns]
cat_feats = [c for c in cat_feats_all if c in dfr.columns]

X = dfr[num_feats + cat_feats]
y = dfr["y_log_resid"].astype(float)

In [24]:
# Time-based split: train (60%) / calib (20%) / test (20%)
if "pickup_datetime" in dfr and dfr["pickup_datetime"].notna().any():
    dfr = dfr.sort_values("pickup_datetime")
idx = dfr.index.to_list()
n = len(idx)
i_train = idx[: int(0.6*n)]
i_calib = idx[int(0.6*n): int(0.8*n)]
i_test  = idx[int(0.8*n):]

X_tr, y_tr = X.loc[i_train], y.loc[i_train]
X_ca, y_ca = X.loc[i_calib], y.loc[i_calib]
X_te, y_te = X.loc[i_test], y.loc[i_test]
base_tr = dfr.loc[i_train, "base_min"].values
base_ca = dfr.loc[i_calib, "base_min"].values
base_te = dfr.loc[i_test , "base_min"].values
y_true_te = dfr.loc[i_test, "trip_duration_minutes"].values

In [25]:
# Preprocess
pre = ColumnTransformer([
    ("num", SimpleImputer(strategy="median"), num_feats),
    ("cat", Pipeline(steps=[
        ("imp", SimpleImputer(strategy="most_frequent")),
        ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
    ]), cat_feats)
], remainder="drop")

In [27]:
# Train quantile models for log-residual
def q_model(q):
    return Pipeline([("prep", pre),
                     ("hgb", HistGradientBoostingRegressor(loss="quantile",
                                                          quantile=q,
                                                          max_leaf_nodes=31,
                                                          min_samples_leaf=40,
                                                          learning_rate=0.08,
                                                          random_state=42))])

m_lo  = q_model(0.10)  # 10th percentile residual
m_med = q_model(0.50)  # median residual (robust point)
m_hi  = q_model(0.90)  # 90th percentile residual

m_lo.fit(X_tr, y_tr)
m_med.fit(X_tr, y_tr)
m_hi.fit(X_tr, y_tr)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='median'),
                                                  ['route_length_km', 'turns',
                                                   'pu_lanes', 'do_lanes',
                                                   'hour_of_day',
                                                   'day_of_week']),
                                                 ('cat',
                                                  Pipeline(steps=[('imp',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['pu_highway',
                                                   'do_highway'])])),
                ('hgb',
                 HistGradientBoostingRegressor(learning_rate=0.08,
                                               loss='quantile',
                                               min_samples_leaf=40,
                                               quantile=0.9,
                                               random_state=42))])

In [28]:
# Conformal calibration on calib split (CQR)
res_lo_ca = m_lo.predict(X_ca)
res_hi_ca = m_hi.predict(X_ca)

# Convert back to minutes for nonconformity
y_lo_ca = base_ca * np.exp(res_lo_ca)
y_hi_ca = base_ca * np.exp(res_hi_ca)
y_ca_minutes = dfr.loc[i_calib, "trip_duration_minutes"].values

# Nonconformity scores: how far outside the (lo,hi) band
s = np.maximum(y_lo_ca - y_ca_minutes, y_ca_minutes - y_hi_ca)
s = np.clip(s, 0, None)

alpha = 0.10  # target 90% coverage
q_conf = np.quantile(s, 1 - alpha)  # conformal offset

In [29]:
# Evaluate on test
res_med_te = m_med.predict(X_te)
res_lo_te  = m_lo.predict(X_te)
res_hi_te  = m_hi.predict(X_te)

y_pred_med = base_te * np.exp(res_med_te)  # point (median)
y_lo = (base_te * np.exp(res_lo_te)) - q_conf
y_hi = (base_te * np.exp(res_hi_te)) + q_conf
y_lo = np.clip(y_lo, 0, None)

# Metrics
abs_err = np.abs(y_true_te - y_pred_med)
mae   = mean_absolute_error(y_true_te, y_pred_med)
medae = median_absolute_error(y_true_te, y_pred_med)
rmse  = np.sqrt(np.mean((y_true_te - y_pred_med)**2))
r2    = r2_score(y_true_te, y_pred_med)
q90   = np.quantile(abs_err, 0.90)
cvar90 = abs_err[abs_err >= q90].mean()

print(f"[Robust eval] MAE={mae:.2f} min | MedAE={medae:.2f} | RMSE={rmse:.2f} | R²={r2:.3f}")
print(f"             CVaR@90(abs err)={cvar90:.2f} min")

# Bucket MAE by true duration (short/medium/long)
bins = [0, 5, 30, np.inf]; labels = ["short","medium","long"]
bucket = pd.cut(y_true_te, bins=bins, labels=labels, right=False, include_lowest=True)
print("Bucket MAE:")
print(pd.Series(abs_err).groupby(bucket).mean().reindex(labels))

# Interval coverage and width
coverage = ((y_true_te >= y_lo) & (y_true_te <= y_hi)).mean()
avg_width = (y_hi - y_lo).mean()
print(f"Conformal PI (90%) coverage={coverage:.3f} | avg width={avg_width:.2f} min")

[Robust eval] MAE=13.65 min | MedAE=3.17 | RMSE=33.31 | R²=-0.106
             CVaR@90(abs err)=95.00 min
Bucket MAE:
short      3.634456
medium     4.131980
long      82.597819
dtype: float64
Conformal PI (90%) coverage=0.866 | avg width=49.75 min


/tmp/ipython-input-2484730085.py:27: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(pd.Series(abs_err).groupby(bucket).mean().reindex(labels))


In [30]:
# Robust predict helper
def _compute_base_min(df_):
    df_ = df_.copy()
    if "route_length_km" not in df_.columns and "route_len_m" in df_.columns:
        df_["route_length_km"] = df_["route_len_m"] / 1000.0
    if "route_time_min" in df_.columns and df_["route_time_min"].notna().any():
        base = df_["route_time_min"].astype(float)
        # fallback where route_time_min is NaN
        base = base.fillna((df_["route_length_km"] / 20.0) * 60.0)
    else:
        base = (df_["route_length_km"] / 20.0) * 60.0
    return base.values

def _ensure_time_feats(df_):
    df_ = df_.copy()
    if "pickup_datetime" in df_.columns:
        df_["pickup_datetime"] = pd.to_datetime(df_["pickup_datetime"], errors="coerce")
    if "hour_of_day" not in df_.columns and "pickup_datetime" in df_.columns:
        df_["hour_of_day"] = df_["pickup_datetime"].dt.hour
    if "day_of_week" not in df_.columns and "pickup_datetime" in df_.columns:
        df_["day_of_week"] = df_["pickup_datetime"].dt.dayofweek
    return df_

def robust_predict(new_df):
    """
    Returns a DataFrame with:
      - y_p50_min : median ETA (minutes)
      - y_p10_min : lower bound of conformal 90% PI
      - y_p90_min : upper bound of conformal 90% PI
    """
    T = _ensure_time_feats(new_df)
    B = _compute_base_min(T)
    X_new = T[num_feats + cat_feats].copy()

    res50 = m_med.predict(X_new)
    res10 = m_lo.predict(X_new)
    res90 = m_hi.predict(X_new)

    y50 = B * np.exp(res50)
    y10 = np.clip(B * np.exp(res10) - q_conf, 0, None)
    y90 = B * np.exp(res90) + q_conf

    return pd.DataFrame({"y_p50_min": y50, "y_p10_min": y10, "y_p90_min": y90}, index=new_df.index)

# Example:
preds = robust_predict(df.head(5))
preds

,y_p50_min,y_p10_min,y_p90_min
64374,6.717689,1.093021,95.925618
123113,5.141671,0.255557,39.433542
194419,7.324004,0.824252,59.551454
260573,4.163512,0.000000,35.408151
276552,4.730529,2.051758,76.424023
